# Fault detection (Research)

Here we will demonstrate the research process of fault detection models. We will train multiple 2D and 3D models on all posible combinations of seismic cubes.

In [ ]:
import sys
import os
from copy import copy
import itertools

import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm_notebook


sys.path.append('../../..')

from seismiqb import *

from seismiqb.batchflow import FilesIndex, Pipeline
from seismiqb.batchflow.research import Option, Research, RP, RC, RD, REP, KV, RI
from seismiqb.batchflow.models.torch import EncoderDecoder, ResBlock
from seismiqb.batchflow import D, B, V, P, R, L, W, C

Here we describe model configuration. In general it is UNet-like architecture where we fix some parameters but all of them are also is a subject of research.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Dice(nn.Module):
    def forward(self, input, target):
        input = torch.sigmoid(input)
        dice_coeff = 2. * (input * target).sum() / (input.sum() + target.sum() + 1e-7)
        return 1 - dice_coeff

ITERS = 2000
BATCH_SIZE = 96
FILTERS = [64, 96, 128, 192, 256]

MODEL_CONFIG = {
    # Model layout
    'initial_block': {
        'base_block': ResBlock,
        'filters': FILTERS[0] // 2,
        'kernel_size': 5,
        'downsample': False,
        'attention': 'scse'
    },

    'body/encoder': {
        'num_stages': 4,
        'order': 'sbd',
        'blocks': {
            'base': ResBlock,
            'n_reps': 1,
            'filters': FILTERS[:-1],
            'attention': 'scse',
        },
    },
    'body/embedding': {
        'base': ResBlock,
        'n_reps': 1,
        'filters': FILTERS[-1],
        'attention': 'scse',
    },
    'body/decoder': {
        'num_stages': 4,
        'upsample': {
            'layout': 'tna',
            'kernel_size': 2,
        },
        'blocks': {
            'base': ResBlock,
            'filters': FILTERS[-2::-1],
            'attention': 'scse',
        },
    },
    'head': {
        'base_block': ResBlock,
        'filters': [16, 8],
        'attention': 'scse'
    },
    'output': torch.sigmoid,
    # Train configuration
    'loss': Dice(),
    'optimizer': {'name': 'Adam', 'lr': 0.005,},
    "decay": {'name': 'exp', 'gamma': 0.1, 'frequency': 150},
    'microbatch': 8,
    'common/activation': 'relu6',
}

The whole training process is described by the following pipeline. We will vary crop shape (2D (1, 128, 256) and 3D (32, 128, 256)) so define it as a `C('crop')` to use with `Research` from `batchflow`.

In [ ]:
train_pipeline = (
    Pipeline()
    # Initialize pipeline variables and model
    .init_variable('loss_history', [])
    .init_model('dynamic', EncoderDecoder, 'model', MODEL_CONFIG)

    # Load data/masks
    .crop(points=D('train_sampler')(BATCH_SIZE), shape=C('crop'), side_view=False)
    .create_masks(dst='masks', width=1)
    .mask_rebatch(src='masks', threshold=0.5, axis=(0, 1))
    .load_cubes(dst='images')
    .adaptive_reshape(src=['images', 'masks'], shape=C('crop'))
    .scale(mode='q', src='images')

    # Augmentations
    .transpose(src=['images', 'masks'], order=(1, 2, 0))
    .flip(axis=1, src=['images', 'masks'], seed=P(R('uniform', 0, 1)), p=0.3)
    .additive_noise(scale=0.005, src='images', dst='images', p=0.3)
    .rotate(angle=P(R('uniform', -15, 15)),
            src=['images', 'masks'], p=0.3)
    .scale_2d(scale=P(R('uniform', 0.85, 1.15)),
              src=['images', 'masks'], p=0.3)
    .transpose(src=['images', 'masks'], order=(2, 0, 1))

    # Training
    .train_model('model',
                 fetches='loss',
                 images=B('images'),
                 masks=B('masks'),
                 save_to=V('loss_history', mode='w'))
    .run_later(D('size'), n_iters=ITERS)
)

Here we describe two callables: to create dataset from paths to cubes and to dump the resulting model.

In [ ]:
def create_dataset(pipeline, config):
    paths = list(config.config()['paths'])
    dataset = SeismicCubeset(FilesIndex(path=paths, no_ext=True))
    dataset.load(label_dir={
        'amplitudes_01_ETP': '/INPUTS/FAULTS/NPY/*',
        'amplitudes_16_PSDM': '/INPUTS/FAULTS/NPY/*',
    }, labels_class=Fault, transform=True, verify=True)
    dataset.modify_sampler(dst='train_sampler', finish=True, low=0.0, high=1.0)
    pipeline.set_dataset(dataset)

In [ ]:
def dump_model(pipeline, path, iteration):
    path = os.path.join(path, 'model_'+str(iteration))
    pipeline.save_model_now('model', path)

Now we construct all possible combinations of 4 availiable cubes.

In [ ]:
datasets = ['/data/seismic_data/seismic_interpretation/CUBE_01_ETP/amplitudes_01_ETP.hdf5',
            '/data/seismic_data/seismic_interpretation/CUBE_16_PSDM/amplitudes_16_PSDM.hdf5']

In [ ]:
dataset = SeismicCubeset(FilesIndex(path=datasets[1:], no_ext=True))

dataset.load(label_dir={
    'amplitudes_01_ETP': '/INPUTS/FAULTS/NPY/*',
    'amplitudes_16_PSDM': '/INPUTS/FAULTS/NPY/*',
}, labels_class=Fault, transform=True, verify=True)
dataset.modify_sampler(dst='train_sampler', finish=True, low=0.0, high=1.0)

for i in range(len(dataset)):
    bounds = min([fault.points[:, 2].min() for fault in dataset.labels[i]]), max([fault.points[:, 2].max() for fault in dataset.labels[i]])
    points = np.random.choice(len(dataset.labels[i]), 3, replace=False)
    for p in points:
        dataset.show_slide(dataset.labels[i][p].points[0, 0], idx=i,
                           figsize=(10,10), zoom_slice = (slice(None, None), slice(*bounds)))

In [ ]:
datasets = [list(itertools.combinations(datasets, i+1)) for i in range(2)]
datasets = sum(datasets, [])
datasets = [KV(item, '_'.join([os.path.splitext(os.path.basename(name))[0][11:] for name in item])) for item in datasets]
datasets

Finally, define domain of parameters: paths to cubes and crop_shapes.

In [ ]:
domain = Option('paths', datasets) * Option('crop', [
    (1, 64, 128), (1, 64, 192), (1, 64, 256), (1, 64, 512),
    (1, 128, 128), (1, 128, 192), (1, 128, 256), (1, 128, 512),
    (1, 256, 256)
])

Describe experiment plan as a research-pipeline. We well use 8 availiable gpus to train models in parallel to increase the speed several times.

In [ ]:
# ! rm -r research_1d_without_transpose

research = (Research()
    .init_domain(domain)
    .add_callable(create_dataset, pipeline=RP('train'), config=RC(), execute='#0')
    .add_pipeline(train_pipeline, name='train', variables='loss_history', logging=True)
    .add_callable(dump_model, pipeline=RP('train'), path=REP(), iteration=RI(), execute=[100, 'last'], logging=True)
)

research.run(name='research_1d_without_transpose', workers=6, devices=[0, 1, 2, 3, 4, 5], timeout=10)

Plot loss functions of all models.

In [ ]:
results = research.load_results().df
(results
 .pivot(index='iteration', columns='sample_index', values='loss_history')
 .rolling(100).mean()
 .plot(xlim=(100, 2000), legend=True, title='loss')
)

In [ ]:
sample_index = results.groupby('sample_index').apply(lambda x: x.iloc[-1]['loss_history']).idxmin()
results[results.sample_index == sample_index]